In [5]:
import os
import bs4

from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.chat_message_histories import ChatMessageHistory

from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain_huggingface import HuggingFaceEmbeddings

from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_groq import ChatGroq

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

from dotenv import load_dotenv

In [7]:
# Carregar variáveis de ambiente
load_dotenv()
api_key = os.environ["GROQ_API_KEY"]

# Instanciar o modelo LLM (Large Language Model)
llm = ChatGroq(model="llama3-8b-8192", api_key=api_key)

In [8]:
# Carregar documento PDF
loader = PyPDFLoader("./pdfs/curriculo.pdf")
docs = loader.load()

In [9]:
# Dividir o texto em pedaços menores
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Indexar os documentos usando FAISS
vectorstore = FAISS.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

# Configurar o recuperador de documentos
retriever = vectorstore.as_retriever()

In [10]:
# Prompt para contextualizar perguntas com histórico de chat
contextualize_q_system_prompt = (
    "Considerando um histórico de bate-papo e a última pergunta do usuário "
    "que pode fazer referência ao contexto no histórico do chat, "
    "formule uma pergunta independente que possa ser entendida "
    "sem o histórico de bate-papo. NÃO responda à pergunta, "
    "apenas reformule-a se necessário e devolva-a como está."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [11]:
# Criar um recuperador consciente do histórico
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [12]:
# Prompt do sistema para perguntas e respostas
system_prompt = (
    "Você é um assistente para tarefas de resposta a perguntas. "
    "Use as seguintes partes do contexto recuperado para responder "
    "a pergunta. Se você não sabe a resposta, diga que você "
    "não sabe. Use no máximo três frases e mantenha a "
    "resposta concisa."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

In [13]:
# Criar a cadeia de perguntas e respostas (QA)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Criar a cadeia de recuperação e geração de respostas (RAG)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [14]:
# Armazenar históricos de conversas
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [15]:
# Executar cadeia de recuperação com histórico de mensagens
conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [16]:
# Fazer consultas e recuperar respostas com histórico de conversa
conversational_rag_chain.invoke(
    {"input": "Do que se trata este documento?"},
    config={"configurable": {"session_id": "abc123"}}
)["answer"]

'Este documento é um currículo vitae (CV) de João Pedro Forequi de Oliveira, um profissional experiente em arquitetura de soluções, com especialização em Data & Analytics.'

In [17]:
conversational_rag_chain.invoke(
    {"input": "Qual é a formação de João Pedro?"},
    config={"configurable": {"session_id": "abc123"}}
)["answer"]

'João Pedro Forequi de Oliveira é bacharel em Ciência da Computação pela Universidade Vila Velha (2017-2021).'

In [18]:
conversational_rag_chain.invoke(
    {"input": "Quais tecnologias estão no domínio de João Pedro?"},
    config={"configurable": {"session_id": "abc123"}}
)["answer"]

'De acordo com o currículo, João Pedro tem experiência com tecnologias da Microsoft Azure, como Azure Synapse Analytics, Azure Data Factory, Azure Databricks, Azure DevOps, Azure Analysis Services e Power BI, além de conhecimento avançado em Python, SQL e NoSQL.'

In [19]:
# Imprimir o histórico de mensagens da sessão
for message in store["abc123"].messages:
    prefix = "AI" if isinstance(message, AIMessage) else "User"
    print(f"{prefix}: {message.content}\n")

User: Do que se trata este documento?

AI: Este documento é um currículo vitae (CV) de João Pedro Forequi de Oliveira, um profissional experiente em arquitetura de soluções, com especialização em Data & Analytics.

User: Qual é a formação de João Pedro?

AI: João Pedro Forequi de Oliveira é bacharel em Ciência da Computação pela Universidade Vila Velha (2017-2021).

User: Quais tecnologias estão no domínio de João Pedro?

AI: De acordo com o currículo, João Pedro tem experiência com tecnologias da Microsoft Azure, como Azure Synapse Analytics, Azure Data Factory, Azure Databricks, Azure DevOps, Azure Analysis Services e Power BI, além de conhecimento avançado em Python, SQL e NoSQL.

